In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. Подготовка данных
data = pd.read_csv('path/to/your/data.csv')
articles = pd.read_csv('path/to/articles.csv')

# Преобразование данных в формат последовательностей
sequences = data.groupby('customer_id')['article_id'].apply(list).reset_index()

# 2. Создание датасета
class RecommendationDataset(Dataset):
    def __init__(self, sequences, max_len):
        self.sequences = sequences
        self.max_len = max_len

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = self.sequences[idx]
        input_seq = sequence[:-1]
        target_seq = sequence[1:]
        return torch.tensor(input_seq, dtype=torch.long), torch.tensor(target_seq, dtype=torch.long)

max_len = 50  # Максимальная длина последовательности
train_sequences, test_sequences = train_test_split(sequences['article_id'].tolist(), test_size=0.2)
train_dataset = RecommendationDataset(train_sequences, max_len)
test_dataset = RecommendationDataset(test_sequences, max_len)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 3. Создание модели Transformer
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, max_len):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.transformer = nn.Transformer(d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward)
        self.fc = nn.Linear(d_model, vocab_size)
        self.max_len = max_len

    def forward(self, src, tgt):
        src = self.embedding(src) * math.sqrt(self.max_len)
        tgt = self.embedding(tgt) * math.sqrt(self.max_len)
        output = self.transformer(src, tgt)
        output = self.fc(output)
        return output

vocab_size = len(articles['article_id'].unique())
d_model = 512
nhead = 8
num_encoder_layers = 6
num_decoder_layers = 6
dim_feedforward = 2048

model = TransformerModel(vocab_size, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, max_len)

# 4. Обучение модели
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for input_seq, target_seq in train_loader:
        optimizer.zero_grad()
        output = model(input_seq, target_seq)
        loss = criterion(output.view(-1, vocab_size), target_seq.view(-1))
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# 5. Оценка модели
model.eval()
total_loss = 0
with torch.no_grad():
    for input_seq, target_seq in test_loader:
        output = model(input_seq, target_seq)
        loss = criterion(output.view(-1, vocab_size), target_seq.view(-1))
        total_loss += loss.item()
print(f'Test Loss: {total_loss / len(test_loader)}')